## Problem 3: How far did people travel? (8 points)

During this task, the aim is to calculate the (air-line) distance in meters that each social media user in the data set prepared in *Problem 2* has travelled in-between the posts. We’re interested in the Euclidean distance between subsequent points generated by the same user.

For this, we will need to use the `userid` column of the data set `kruger_posts.shp` that we created in *Problem 2*.

Answer the following questions:
- What was the shortest distance a user travelled between all their posts (in meters)?
- What was the mean distance travelled per user (in meters)?
- What was the maximum distance a user travelled (in meters)?

---


### a) Read the input file and re-project it

- Read the input file `kruger_points.shp` into a geo-data frame `kruger_points`
- Transform the data from WGS84 to an `EPSG:32735` projection (UTM Zone 35S, suitable for South Africa). This CRS has *metres* as units.

In [1]:
# ADD YOUR OWN CODE HERE
import geopandas as gpd

kruger_points = gpd.read_file("data/kruger_points.shp")
kruger_points = kruger_points.to_crs("EPSG:32735")

In [7]:
kruger_points

,lat,lon,timestamp,userid,geometry
0,-24.980792,31.484633,2015-07-07 03:02,66487960,POINT (952912.89 7229683.258)
1,-25.499225,31.508906,2015-07-07 03:18,65281761,POINT (953433.223 7172080.632)
2,-24.342578,30.930866,2015-03-07 03:38,90916112,POINT (898955.144 7302197.408)
3,-24.854614,31.519718,2015-10-07 05:04,37959089,POINT (956927.218 7243564.942)
4,-24.921069,31.520836,2015-10-07 05:19,27793716,POINT (956794.955 7236187.926)
...,...,...,...,...,...
81374,-24.799541,31.354469,2015-09-05 02:23,90744213,POINT (940395.058 7250220.045)
81375,-25.467992,30.956033,2015-02-05 02:40,71109799,POINT (897877.38 7177317.787)
81376,-25.332223,30.997409,2015-08-05 02:40,54796261,POINT (902494.313 7192250.813)
81377,-25.508851,31.005536,2015-08-05 02:43,78762204,POINT (902724.67 7172637.035)


In [2]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
kruger_points.head()

,lat,lon,timestamp,userid,geometry
0,-24.980792,31.484633,2015-07-07 03:02,66487960,POINT (952912.89 7229683.258)
1,-25.499225,31.508906,2015-07-07 03:18,65281761,POINT (953433.223 7172080.632)
2,-24.342578,30.930866,2015-03-07 03:38,90916112,POINT (898955.144 7302197.408)
3,-24.854614,31.519718,2015-10-07 05:04,37959089,POINT (956927.218 7243564.942)
4,-24.921069,31.520836,2015-10-07 05:19,27793716,POINT (956794.955 7236187.926)


In [3]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
import pyproj
assert kruger_points.crs == pyproj.CRS("EPSG:32735")

In [4]:
kruger_points.crs

<Projected CRS: EPSG:32735>
Name: WGS 84 / UTM zone 35S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 24°E and 30°E, southern hemisphere between 80°S and equator, onshore and offshore. Botswana. Burundi. Democratic Republic of the Congo (Zaire). Rwanda. South Africa. Tanzania. Uganda. Zambia. Zimbabwe.
- bounds: (24.0, -80.0, 30.0, 0.0)
Coordinate Operation:
- name: UTM zone 35S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### b) Group the data by user id

Group the data by `userid` and store the grouped data in a variable `grouped_by_users`

In [ ]:
# ADD YOUR OWN CODE HERE

In [6]:
kruger_points["userid"].nunique()

14990

In [8]:
grouped_by_users = kruger_points.groupby("userid")

In [9]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the number of groups:
assert len(grouped_by_users.groups) == kruger_points["userid"].nunique(), "Number of groups should match number of unique users!"

### c) Create `shapely.geometry.LineString` objects for each user connecting the points from oldest to most recent

There are multiple ways to solve this problem (see the [hints for this exercise](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-2/exercise-2.html). You can use, for instance, a dictionary or an empty GeoDataFrame to collect data that is generated using the steps below:

- Use a for-loop to iterate over the grouped object. For each user’s data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a `shapely.geometry.LineString` based on the user’s points

**CAREFUL**: Remember that every LineString needs at least two points. Skip users who have less than two posts.

Store the results in a `geopandas.GeoDataFrame` called `movements`, and remember to assign a CRS.

In [11]:
# ADD YOUR OWN CODE HERE
from shapely.geometry import LineString
import pandas as pd

# Initialize an empty list to collect data
movements_list = []

# Iterate over each user group
for userid, group in grouped_by_users:
    # Sort the rows by timestamp
    group = group.sort_values('timestamp')
    
    # Create a LineString if there are at least two points
    if len(group) > 1:
        line = LineString(group.geometry.tolist())
        movements_list.append({'userid': userid, 'geometry': line})

# Convert the list to a GeoDataFrame
movements = gpd.GeoDataFrame(movements_list, crs='EPSG:32735')

In [12]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the result
print(type(movements))
print(movements.crs)

movements

<class 'geopandas.geodataframe.GeoDataFrame'>
EPSG:32735


,userid,geometry
0,16301,"LINESTRING (942231.63 7254606.868, 938934.725 ..."
1,45136,"LINESTRING (905394.5 7193375.148, 905394.5 719..."
2,50136,"LINESTRING (944551.607 7253384.183, 963788.403..."
3,88775,"LINESTRING (902800.817 7192546.975, 902800.839..."
4,88918,"LINESTRING (959332.961 7219877.715, 963788.403..."
...,...,...
9021,99921781,"LINESTRING (902885.19 7196931.096, 904027.71 7..."
9022,99936874,"LINESTRING (963782.211 7228000.079, 963754.402..."
9023,99964140,"LINESTRING (938876.653 7305143.369, 938876.943..."
9024,99986933,"LINESTRING (935937.029 7305973.536, 936598.681..."


### d) Calculate the distance between all posts of a user

- Check once more that the CRS of the data frame is correct
- Compute the lengths of the lines, and store it in a new column called `distance`

In [13]:
# ADD YOUR OWN CODE HERE
movements = movements.set_crs(epsg=32735)

movements["distance"] = movements.length

In [14]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,userid,geometry,distance
0,16301,"LINESTRING (942231.63 7254606.868, 938934.725 ...",328455.115430
1,45136,"LINESTRING (905394.5 7193375.148, 905394.5 719...",0.000000
2,50136,"LINESTRING (944551.607 7253384.183, 963788.403...",159189.081019
3,88775,"LINESTRING (902800.817 7192546.975, 902800.839...",0.080245
4,88918,"LINESTRING (959332.961 7219877.715, 963788.403...",9277.252211


### e) Answer the original questions

You should now be able to quickly find answers to the following questions: 
- What was the shortest distance a user travelled between all their posts (in meters)? (store the value in a variable `shortest_distance`)
- What was the mean distance travelled per user (in meters)? (store the value in a variable `mean_distance`)
- What was the maximum distance a user travelled (in meters)? (store the value in a variable `longest_distance`)

In [16]:
# ADD YOUR OWN CODE HERE
shortest_distance = movements["distance"].min()
shortest_distance

np.float64(0.0)

In [17]:
mean_distance = movements["distance"].mean()
mean_distance

np.float64(107133.51202944393)

In [18]:
longest_distance = movements["distance"].max()
longest_distance

np.float64(6970668.816343964)

### f) Save the movements in a file

Save the `movements` into a new Shapefile called `movements.shp` inside the `data` directory.

In [25]:
# ADD YOUR OWN CODE HERE
import pathlib
pathlib.Path()

path = pathlib.Path().resolve()

DATA_DIRECTORY = path / "data"
DATA_DIRECTORY

movements.to_file(DATA_DIRECTORY / "movements.shp")

In [26]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

assert (DATA_DIRECTORY / "movements.shp").exists()


---

# Fantastic job!

That’s all for this week! 